In [1]:
import sys, ast
from typing import Dict, List, Tuple
import matplotlib
import matplotlib.pyplot as plt
import collections
import random
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.tensorboard import SummaryWriter


is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()


In [2]:

import requests, random, json

url = 'http://127.0.0.1:5000'
mapping = {"STOP": "0x0", "ADD": "0x1", "MUL": "0x2", "SUB": "0x3", "DIV": "0x4", "SDIV": "0x5", "MOD": "0x6",
               "SMOD": "0x7", "ADDMOD": "0x8", "MULMOD": "0x9", "EXP": "0xa", "SIGNEXTEND": "0xb", "LT": "0x10",
               "GT": "0x11", "SLT": "0x12", "SGT": "0x13", "EQ": "0x14", "ISZERO": "0x15", "AND": "0x16", "OR": "0x17",
               "XOR": "0x18", "NOT": "0x19", "BYTE": "0x1a", "SHL": "0x1b", "SHR": "0x1c", "SAR": "0x1d",
               "SHA3": "0x20", "ADDRESS": "0x30", "BALANCE": "0x31", "ORIGIN": "0x32", "CALLER": "0x33",
               "CALLVALUE": "0x34", "CALLDATALOAD": "0x35", "CALLDATASIZE": "0x36", "CALLDATACOPY": "0x37",
               "CODESIZE": "0x38", "CODECOPY": "0x39", "GASPRICE": "0x3a", "EXTCODESIZE": "0x3b", "EXTCODECOPY": "0x3c",
               "RETURNDATASIZE": "0x3d", "RETURNDATACOPY": "0x3e", "EXTCODEHASH": "0x3f", "BLOCKHASH": "0x40",
               "COINBASE": "0x41", "TIMESTAMP": "0x42", "NUMBER": "0x43", "DIFFICULTY": "0x44", "GASLIMIT": "0x45",
               "CHAINID": "0x46", "SELFBALANCE": "0x47", "BASEFEE": "0x48", "POP": "0x50", "MLOAD": "0x51",
               "MSTORE": "0x52", "MSTORE8": "0x53", "SLOAD": "0x54", "SSTORE": "0x55", "JUMP": "0x56", "JUMPI": "0x57",
               "PC": "0x58", "MSIZE": "0x59", "GAS": "0x5a", "JUMPDEST": "0x5b", "PUSH1": "0x60", "PUSH2": "0x61",
               "PUSH3": "0x62", "PUSH4": "0x63", "PUSH5": "0x64", "PUSH6": "0x65", "PUSH7": "0x66", "PUSH8": "0x67",
               "PUSH9": "0x68", "PUSH10": "0x69", "PUSH11": "0x6a", "PUSH12": "0x6b", "PUSH13": "0x6c",
               "PUSH14": "0x6d", "PUSH15": "0x6e", "PUSH16": "0x6f", "PUSH17": "0x70", "PUSH18": "0x71",
               "PUSH19": "0x72", "PUSH20": "0x73", "PUSH21": "0x74", "PUSH22": "0x75", "PUSH23": "0x76",
               "PUSH24": "0x77", "PUSH25": "0x78", "PUSH26": "0x79", "PUSH27": "0x7a", "PUSH28": "0x7b",
               "PUSH29": "0x7c", "PUSH30": "0x7d", "PUSH31": "0x7e", "PUSH32": "0x7f", "DUP1": "0x80", "DUP2": "0x81",
               "DUP3": "0x82", "DUP4": "0x83", "DUP5": "0x84", "DUP6": "0x85", "DUP7": "0x86", "DUP8": "0x87",
               "DUP9": "0x88", "DUP10": "0x89", "DUP11": "0x8a", "DUP12": "0x8b", "DUP13": "0x8c", "DUP14": "0x8d",
               "DUP15": "0x8e", "DUP16": "0x8f", "SWAP1": "0x90", "SWAP2": "0x91", "SWAP3": "0x92", "SWAP4": "0x93",
               "SWAP5": "0x94", "SWAP6": "0x95", "SWAP7": "0x96", "SWAP8": "0x97", "SWAP9": "0x98", "SWAP10": "0x99",
               "SWAP11": "0x9a", "SWAP12": "0x9b", "SWAP13": "0x9c", "SWAP14": "0x9d", "SWAP15": "0x9e",
               "SWAP16": "0x9f", "LOG0": "0xa0", "LOG1": "0xa1", "LOG2": "0xa2", "LOG3": "0xa3", "LOG4": "0xa4",
               "PUSH": "0xb0", "DUP": "0xb1", "SWAP": "0xb2", "CREATE": "0xf0", "CALL": "0xf1", "CALLCODE": "0xf2",
               "RETURN": "0xf3", "DELEGATECALL": "0xf4", "STATICCALL": "0xfa", "REVERT": "0xfd", "INVALID": "0xfe",
               "SELFDESTRUCT": "0xff"}


def compile_sol(solfile):
    r = requests.post(url + '/compile', data={'solfile': solfile})



def deploy(contract, deploy_on_address):
    if "Attack" in contract:
        r = requests.post(url + '/deploy', data={'contract': contract,'deploy_on_address':deploy_on_address})
    else:
        r = requests.post(url + '/deploy', data={'contract': contract})
    if r.status_code != 200:
        return False
    return r.json()


def execute(contract, function):
    r = requests.post(url + '/execute', data={'contract': contract, 'function': function})

    if r.status_code != 200:
        # print(r.text)
        return False
    return r.json()
def get_contract():
    
    r = requests.post(url + '/get_contracts')

    if r.status_code != 200:
        return False
    return r.json()
def get_groups_function(contract):
    r = requests.post(url + '/get_groups_functions', data={'contract': contract})

    if r.status_code != 200:
        return False
    return r.json()

def compile_deploy():
    global contract_ne, funtions_ne, contract_
    compile_sol('flag_check.sol')
    contracts = get_contract()
    print(contracts)

    
    # contract = random.choice(contracts)
    # contract_ne = contract
    a = deploy("DepositFunds","")
    print(a)
    b = deploy("Attack",a['address'])
    
    contract_ = contracts[0]
    # groups_function = deploy(contract)

    # if (not groups_function): return (-1, -1)
    aaaaa = get_groups_function("Attack")
    
    
    
    # function = groups_function['Groups']
    funtions_ne = aaaaa

    
    # print(function)
    return contract_, funtions_ne

def check_get_state(mapping):
    contract, function = compile_deploy()
    while True:
        number = str(random.randint(0,2))
        elements, rewarded, rbugs = get_state(contract, function,number, mapping)
        if elements != -1 and rewarded != -1:  # execute OK
            # print("EXECUTE OK")
            return elements, rewarded, rbugs
        else:
            print("ERROR")

In [3]:
# r = requests.post(url + '/compile', data={'solfile': "1.sol"})

# r = requests.post(url + '/get_contracts')



# sasas = r.json()['result']
# x = ast.literal_eval(sasas)
# x

In [4]:
# r = requests.post(url + '/compile', data={'solfile': "1.sol"})
# a = deploy("DepositFunds","kkk")
# b = deploy("Attack",a)
# execute("Attack", "attack_deposit")
# # execute("Attack", "attack_withdraw")


In [5]:
# a = deploy("DepositFunds","kkk")
# b = deploy("Attack",a)
    

In [6]:
# aaaaa = get_groups_function("Attack")
# aaaaa

In [7]:
def get_state(contract, function,stt,mapping):
    
    choose = None

    # while True:
        # number = str(random.randint(0, 4))
    print(function, 'stt ne ',stt)
    function_list = json.loads(json.dumps(function))[str(stt)]
    # print('ddwwdw',(function_list))
    if (len(function_list) > 0):
        choose = random.choice(function_list)
        print(choose)
    else:
        # chỗ này cần cộng cái state cho 1 array full 0 và reward = 0
        print("[+] TH ma gr khong co phan tu nao")
        return [0]*340 , -0.1, [0]
    

    result = execute(contract, choose)
    # if not result: 
    #     # print(result)
    #     return [0]*340 , -0.25, [0]
    # print(result)
    elements = []
    # print(result)
    F = json.loads(result['F'])

    for k, v in F.items():
        if (type(v) == type(list())):
            for element in v:
                if (type(element)) == type(str()):
                    elements += [int(mapping[element], 16)]
                else:
                    elements += [element]
        else:
            elements += [v]
    # print(elements)

    SC = json.loads(result['SC'])
    for k, v in SC.items():
        if (type(v) == type(dict())):
            for k_, v_ in v.items():
                elements += [v_]
        else:
            elements += [v]

    elements += [result["Rewards"]]
    # print((elements))
    rewarded = elements.pop()
    rbugs = result['Rbugs']
    # print(rbugs)
    return elements, rewarded, rbugs

In [8]:
# Q_network
class Q_net(nn.Module):
    def __init__(self, state_space=None,
                 action_space=None):
        super(Q_net, self).__init__()

        # space size check
        assert state_space is not None, "None state_space input: state_space should be selected."
        assert action_space is not None, "None action_space input: action_space should be selected."

        self.hidden_space = 64
        self.state_space = state_space
        self.action_space = action_space

        self.Linear1 = nn.Linear(self.state_space, self.hidden_space)
        self.lstm    = nn.LSTM(self.hidden_space,self.hidden_space, batch_first=True)
        self.Linear2 = nn.Linear(self.hidden_space, self.action_space)

    def forward(self, x, h, c):
        x = F.relu(self.Linear1(x))
        x, (new_h, new_c) = self.lstm(x,(h,c))
        x = self.Linear2(x)
        return x, new_h, new_c

    def sample_action(self, obs, h,c, epsilon,boolne):
        
        output = self.forward(obs, h,c)
        aaa = random.randint(0,2)
        if boolne ==True:
             return output[0].argmax().item(), output[1] , output[2]
        print("gia tri cua aaa",aaa)
        if aaa  < epsilon:
            print("vao dk if cua ham sample_action")
            return aaa, output[1], output[2]
        else:
            print("vao dk else cua ham sample_action")
            return output[0].argmax().item(), output[1] , output[2]
    
    def init_hidden_state(self, batch_size, training=None):

        assert training is not None, "training step parameter should be dtermined"

        if training is True:
            return torch.zeros([1, batch_size, self.hidden_space]), torch.zeros([1, batch_size, self.hidden_space])
        else:
            return torch.zeros([1, 1, self.hidden_space]), torch.zeros([1, 1, self.hidden_space])


In [9]:
class EpisodeMemory():
    """Episode memory for recurrent agent"""

    def __init__(self, random_update=False, 
                       max_epi_num=100, max_epi_len=500,
                       batch_size=1,
                       lookup_step=None):
        self.random_update = random_update # if False, sequential update
        self.max_epi_num = max_epi_num
        self.max_epi_len = max_epi_len
        self.batch_size = batch_size
        self.lookup_step = lookup_step

        if (random_update is False) and (self.batch_size > 1):
            sys.exit('It is recommend to use 1 batch for sequential update, if you want, erase this code block and modify code')

        self.memory = collections.deque(maxlen=self.max_epi_num)

    def put(self, episode):
        self.memory.append(episode)

    def sample(self):
        sampled_buffer = []

        ##################### RANDOM UPDATE ############################
        if self.random_update: # Random upodate
            sampled_episodes = random.sample(self.memory, self.batch_size)
            
            check_flag = True # check if every sample data to train is larger than batch size
            min_step = self.max_epi_len

            for episode in sampled_episodes:
                min_step = min(min_step, len(episode)) # get minimum step from sampled episodes

            for episode in sampled_episodes:
                if min_step > self.lookup_step: # sample buffer with lookup_step size
                    idx = np.random.randint(0, len(episode)-self.lookup_step+1)
                    sample = episode.sample(random_update=self.random_update, lookup_step=self.lookup_step, idx=idx)
                    sampled_buffer.append(sample)
                else:
                    idx = np.random.randint(0, len(episode)-min_step+1) # sample buffer with minstep size
                    sample = episode.sample(random_update=self.random_update, lookup_step=min_step, idx=idx)
                    sampled_buffer.append(sample)

        ##################### SEQUENTIAL UPDATE ############################           
        else: # Sequential update
           
            idx = np.random.randint(0, len(self.memory))
            sampled_buffer.append(self.memory[idx].sample(random_update=self.random_update))

        return sampled_buffer, len(sampled_buffer[0]['obs']) # buffers, sequence_length

    def __len__(self):
        return len(self.memory)


In [10]:
class EpisodeBuffer:
    """A simple numpy replay buffer."""

    def __init__(self):
        self.obs = []
        self.action = []
        self.reward = []
        self.next_obs = []
        self.done = []

    def put(self, transition):
        self.obs.append(transition[0])
        self.action.append(transition[1])
        self.reward.append(transition[2])
        self.next_obs.append(transition[3])
        self.done.append(transition[4])

    def sample(self, random_update=False, lookup_step=None, idx=None) -> Dict[str, np.ndarray]:
        obs = np.array(self.obs)
        action = np.array(self.action)
        reward = np.array(self.reward)
        next_obs = np.array(self.next_obs)
        done = np.array(self.done)

        if random_update is True:
            obs = obs[idx:idx+lookup_step]
            action = action[idx:idx+lookup_step]
            reward = reward[idx:idx+lookup_step]
            next_obs = next_obs[idx:idx+lookup_step]
            done = done[idx:idx+lookup_step]

        return dict(obs=obs,
                    acts=action,
                    rews=reward,
                    next_obs=next_obs,
                    done=done)

    def __len__(self) -> int:
        return len(self.obs)

In [11]:
def train(q_net=None, target_q_net=None, episode_memory=None,
          device=None, 
          optimizer = None,
          batch_size=1,
          learning_rate=1e-3,
          gamma=0.99):

    assert device is not None, "None Device input: device should be selected."

    # Get batch from replay buffer
    samples, seq_len = episode_memory.sample()

    observations = []
    actions = []
    rewards = []
    next_observations = []
    dones = []

    for i in range(batch_size):
        observations.append(samples[i]["obs"])
        actions.append(samples[i]["acts"][:seq_len])
        rewards.append(samples[i]["rews"])
        next_observations.append(samples[i]["next_obs"])
        dones.append(samples[i]["done"])

    observations = np.array(observations)
    actions = np.array(actions)
    rewards = np.array(rewards)
    next_observations = np.array(next_observations)
    dones = np.array(dones)

    observations = torch.FloatTensor(observations.reshape(batch_size,seq_len,-1)).to(device)
    actions = torch.LongTensor(actions.reshape(batch_size,seq_len,-1)).to(device)
    rewards = torch.FloatTensor(rewards.reshape(batch_size,seq_len,-1)).to(device)
    next_observations = torch.FloatTensor(next_observations.reshape(batch_size,seq_len,-1)).to(device)
    dones = torch.FloatTensor(dones.reshape(batch_size,seq_len,-1)).to(device)

    h_target, c_target = target_q_net.init_hidden_state(batch_size=batch_size, training=True)

    q_target, _, _ = target_q_net(next_observations, h_target.to(device), c_target.to(device))

    q_target_max = q_target.max(2)[0].view(batch_size,seq_len,-1).detach()
    targets = rewards + gamma*q_target_max*dones


    h, c = q_net.init_hidden_state(batch_size=batch_size, training=True)
    q_out, _, _ = q_net(observations, h.to(device), c.to(device))
    q_a = q_out.gather(2, actions)

    # Multiply Importance Sampling weights to loss        
    loss = F.smooth_l1_loss(q_a, targets)
    
    # Update Network
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [12]:
def plot_durations(show_result=False):
    plt.figure(1)
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())

In [13]:
def seed_torch(seed):
        torch.manual_seed(seed)
        if torch.backends.cudnn.enabled:
            torch.backends.cudnn.benchmark = False
            torch.backends.cudnn.deterministic = True

# def save_model(model, path='default.pth'):
#         torch.save(model.state_dict(), path)


In [14]:
if __name__ == "__main__":

    # Env parameters
    model_name = "DRQN_POMDP_Random"
    env_name = "CartPole-v1"
    seed = 1
    exp_num = 'SEED'+'_'+str(seed)

    # Set gym environment
    # env = gym.make(env_name)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Set the seed
    np.random.seed(seed)
    random.seed(seed)
    seed_torch(seed)
    # env.seed(seed)

    # default `log_dir` is "runs" - we'll be more specific here
    writer = SummaryWriter('runs/'+env_name+"_"+model_name+"_"+exp_num)

    # Set parameters
    batch_size = 8
    learning_rate = 1e-3
    buffer_len = int(100000)
    min_epi_num = 20 # Start moment to train the Q network
    episodes = 650
    print_per_iter = 20
    target_update_period = 4
    eps_start = 1.75
    eps_end = 0.001
    eps_decay = 0.995
    tau = 1e-2
    max_step = 2000

    # DRQN param
    random_update = True# If you want to do random update instead of sequential update
    lookup_step = 20 # If you want to do random update instead of sequential update
    max_epi_len = 100 
    max_epi_step = max_step

    

    # Create Q functions
    Q = Q_net(state_space=340, 
              action_space=3).to(device)
    Q_target = Q_net(state_space=340, 
                     action_space=3).to(device)

    Q_target.load_state_dict(Q.state_dict())

    # Set optimizer
    score = 0
    score_sum = 0
    optimizer = optim.Adam(Q.parameters(), lr=learning_rate)

    epsilon = eps_start
    
    episode_memory = EpisodeMemory(random_update=random_update, 
                                   max_epi_num=100, max_epi_len=600, 
                                   batch_size=batch_size, 
                                   lookup_step=lookup_step)

In [15]:
    episode_durations = []
    for i in range(episodes):
        state, info, rbugs = check_get_state(mapping)
        obs = state
        # done = False
        
        episode_record = EpisodeBuffer()
        h, c = Q.init_hidden_state(batch_size=batch_size, training=False)

        for t in range(max_step):
            print("=======  voi lan chay thu",t,"trong episode thu ",i)
            # Get action
            if i >200: 
                a, h, c = Q.sample_action(torch.from_numpy(np.asarray(obs)).float().to(device).unsqueeze(0).unsqueeze(0), 
                                              h.to(device), c.to(device),
                                              epsilon, True)
            else:
                a, h, c = Q.sample_action(torch.from_numpy(np.asarray(obs)).float().to(device).unsqueeze(0).unsqueeze(0), 
                                              h.to(device), c.to(device),
                                              epsilon, False)

            # Do action
            print('stt la ',a)
         
            s_prime, r, done= get_state(contract_, funtions_ne,a, mapping)
            print("++++++++ gia tri cua done la ",done)
            obs_prime = s_prime
            
            if t == 3 and done ==1:
                r += 0.4
            elif t==4 and done ==1:
                r += 0.3
            elif t==5 and done ==1:
                r += 0.1
            
            # make data
            done_mask = done * 1.0

            episode_record.put([obs, a, r, obs_prime, done_mask])

            obs = obs_prime
            
            score += r
            score_sum += r

            if len(episode_memory) >= min_epi_num:
                train(Q, Q_target, episode_memory, device, 
                        optimizer=optimizer,
                        batch_size=batch_size,
                        learning_rate=learning_rate)

                if (t+1) % target_update_period == 0:
                    # Q_target.load_state_dict(Q.state_dict()) <- navie update
                    for target_param, local_param in zip(Q_target.parameters(), Q.parameters()): # <- soft update
                            target_param.data.copy_(tau*local_param.data + (1.0 - tau)*target_param.data)
                            print(2222)
                
            if done == 1:
                print("[+]done")
                episode_durations.append(t + 1)
                plot_durations()
                break
        
        episode_memory.put(episode_record)
        
        epsilon = max(eps_end, epsilon * eps_decay) #Linear annealing

        if i % print_per_iter == 0 and i!=0:
            print("n_episode :{}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(
                                                            i, score_sum/print_per_iter, len(episode_memory), epsilon*100))
            score_sum=0.0
            # save_model(Q, model_name+"_"+exp_num+'.pth')

        # Log the reward
        writer.add_scalar('Rewards per episodes', score, i)
        score = 0
        
    plot_durations(show_result=True)
    plt.ioff()
    plt.show()
        
        
    writer.close()
    # env.close()

['Attack', 'DepositFunds']
{'address': '0x98D80d9319B217C07da816e8E9e6ADF5B46a7178'}
{'0': ['deposit'], '1': ['withdraw'], '2': ['getflag']} stt ne  1
withdraw
=======  voi lan chay thu 0 trong episode thu  20
gia tri cua aaa 1
vao dk if cua ham sample_action
stt la  1
{'0': ['deposit'], '1': ['withdraw'], '2': ['getflag']} stt ne  1
withdraw
++++++++ gia tri cua done la  0
=======  voi lan chay thu 1 trong episode thu  20
gia tri cua aaa 2
vao dk else cua ham sample_action
stt la  0
{'0': ['deposit'], '1': ['withdraw'], '2': ['getflag']} stt ne  0
deposit
++++++++ gia tri cua done la  0
=======  voi lan chay thu 2 trong episode thu  20
gia tri cua aaa 0
vao dk if cua ham sample_action
stt la  0
{'0': ['deposit'], '1': ['withdraw'], '2': ['getflag']} stt ne  0
deposit
++++++++ gia tri cua done la  0
=======  voi lan chay thu 3 trong episode thu  20
gia tri cua aaa 0
vao dk if cua ham sample_action
stt la  0
{'0': ['deposit'], '1': ['withdraw'], '2': ['getflag']} stt ne  0
deposit
+++++

KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>